In [1]:
import pandas as pd
import numpy as np
from itertools import product
from datetime import datetime
import statistics, math

In [2]:
df_stocks = pd.read_csv('../data/amundi-msci-wrld-ae-c.csv',parse_dates=['Date'], index_col=['Date'])
df_corp_bonds = pd.read_csv('../data/ishares-global-corporate-bond-$.csv',parse_dates=['Date'], index_col=['Date'])
df_pub_bonds = pd.read_csv('../data/db-x-trackers-ii-global-sovereign-5.csv',parse_dates=['Date'], index_col=['Date'])
df_gold = pd.read_csv('../data/spdr-gold-trust.csv',parse_dates=['Date'], index_col=['Date'])
df_cash = pd.read_csv('../data/usdollar.csv',parse_dates=['Date'], index_col=['Date'])

In [3]:
def fill_mising_dates (df):
    df.index = pd.to_datetime(df.index, format='%b %d, %Y')
    df = df.sort_values(by = 'Date')

    idx = pd.date_range(start = '2020-01-01', end ='2020-12-31')
    avg_price = df['Price'].mean()
    df = df.reindex(idx, fill_value = round(avg_price,2))
    
    return df

In [4]:
df_stocks = fill_mising_dates(df_stocks)
df_corp_bonds = fill_mising_dates(df_corp_bonds)
df_pub_bonds = fill_mising_dates(df_pub_bonds)
df_gold = fill_mising_dates(df_gold)
df_cash = fill_mising_dates(df_cash)

1.3 Tasks related to investment strategies generation

Portfolio allocation

In [5]:
values = [0,20,40,60,80,100]
results = []

for i in product(values, repeat = 5):
    if sum(i) == 100:
        results.append(i)
        
#print(results)
#print(len(results))

In [6]:
df_portfolio_allocation = pd.DataFrame(results)
df2 = df_portfolio_allocation
df2.to_csv('../data/portfolio_allocations.csv', index = False,header = ['ST', 'CB', 'PB', 'GO', 'CA'])
df_portfolio_allocation = pd.read_csv('../data/portfolio_allocations.csv')
df_portfolio_allocation


,ST,CB,PB,GO,CA
0,0,0,0,0,100
1,0,0,0,20,80
2,0,0,0,40,60
3,0,0,0,60,40
4,0,0,0,80,20
...,...,...,...,...,...
121,80,0,0,0,20
122,80,0,0,20,0
123,80,0,20,0,0
124,80,20,0,0,0


In [7]:
initial_date = input("Select date to calculate(YYYY-MM-DD): ")
#initial_date = '2020-02-04'
return_values = []

for i in range(len(df_portfolio_allocation)):
    buy_amount=0
    curr_val = 0
    if df_portfolio_allocation.loc[i,'ST'] >0:
        buy_amount += (df_portfolio_allocation.loc[i,'ST']/df_stocks.loc[initial_date,'Price'] ) * df_stocks.loc[initial_date,'Price']
        #print(buy_amount)
        curr_val += (df_portfolio_allocation.loc[i,'ST']/df_stocks.loc[initial_date,'Price' ]) * df_stocks.loc['2020-12-31','Price']
        #print(curr_val)

    if df_portfolio_allocation.loc[i,'CB'] >0:
        buy_amount += (df_portfolio_allocation.loc[i,'CB']/df_corp_bonds.loc[initial_date,'Price'] ) * df_corp_bonds.loc[initial_date,'Price']
        #print(buy_amount)
        curr_val += (df_portfolio_allocation.loc[i,'CB']/df_corp_bonds.loc[initial_date,'Price' ]) * df_corp_bonds.loc['2020-12-31','Price']
        #print(curr_val)
    if df_portfolio_allocation.loc[i,'PB'] >0:
        buy_amount += (df_portfolio_allocation.loc[i,'PB']/df_pub_bonds.loc[initial_date,'Price'] ) * df_pub_bonds.loc[initial_date,'Price']
        #print(buy_amount)
        curr_val += (df_portfolio_allocation.loc[i,'PB']/df_pub_bonds.loc[initial_date,'Price' ]) * df_pub_bonds.loc['2020-12-31','Price']
        #print(curr_val)
    if df_portfolio_allocation.loc[i,'GO'] >0:
        buy_amount += (df_portfolio_allocation.loc[i,'GO']/df_gold.loc[initial_date,'Price'] ) * df_gold.loc[initial_date,'Price']
        #print(buy_amount)
        curr_val += (df_portfolio_allocation.loc[i,'GO']/df_gold.loc[initial_date,'Price' ]) * df_gold.loc['2020-12-31','Price']
        #print(curr_val)
    if df_portfolio_allocation.loc[i,'CA'] >0:
        buy_amount += (df_portfolio_allocation.loc[i,'CA']/df_cash.loc[initial_date,'Price'] ) * df_cash.loc[initial_date,'Price']
        #print(buy_amount)
        curr_val += (df_portfolio_allocation.loc[i,'CA']/df_cash.loc[initial_date,'Price' ]) * df_cash.loc['2020-12-31','Price']
        #print(curr_val)

    return_values.append(round(((curr_val-buy_amount)/ buy_amount),3)*100)

df_portfolio_metrics = df_portfolio_allocation
df_portfolio_metrics['RETURN'] = return_values
df_portfolio_metrics

KeyError: ''

In [20]:
testing = pd.DataFrame()
values_per_day = []
for i in df_stocks.index:
    value_per_asset= 0
    for j in range(len(df_portfolio_allocation)):
        if df_portfolio_allocation.loc[j,'ST'] >0:
            value_per_asset += (df_portfolio_allocation.loc[j,'ST']/df_stocks.loc[i,'Price'] ) * df_stocks.loc[i,'Price']

        if df_portfolio_allocation.loc[j,'CB'] >0:
            value_per_asset += (df_portfolio_allocation.loc[j,'CB']/df_corp_bonds.loc[i,'Price'] ) * df_corp_bonds.loc[i,'Price']
        if df_portfolio_allocation.loc[j,'PB'] >0:
            value_per_asset += (df_portfolio_allocation.loc[j,'PB']/df_pub_bonds.loc[i,'Price'] ) * df_pub_bonds.loc[i,'Price']
            
        if df_portfolio_allocation.loc[j,'GO'] >0:
            value_per_asset += (df_portfolio_allocation.loc[j,'GO']/df_gold.loc[i,'Price'] ) * df_gold.loc[i,'Price']
            
        if df_portfolio_allocation.loc[j,'CA'] >0:
            value_per_asset += (df_portfolio_allocation.loc[j,'CA']/df_cash.loc[i,'Price'] ) * df_cash.loc[i,'Price']
    values_per_day.append(value_per_asset)    
           

In [36]:
def std_dev(values):
    accum = 0
    mean_val = statistics.mean(values)
    #print(mean_val)
    for x in values:
        #print(x)   
        accum+=(x-mean_val)**2
        #print(x, statistics.mean(values))
    return math.sqrt(accum/366)


In [38]:

#initial_date = '2020-02-04'
values_per_port = []
for portfolio in range(len(df_portfolio_allocation)):
    values_per_day = []
    
    for date in df_stocks.index:
        value_per_day = 0
        if df_portfolio_allocation.loc[portfolio,'ST'] >0:
            value_per_day += (df_portfolio_allocation.loc[portfolio,'ST']/df_stocks.loc[date,'Price'] ) * df_stocks.loc[date,'Price']

        if df_portfolio_allocation.loc[portfolio,'CB'] >0:
            value_per_day += (df_portfolio_allocation.loc[portfolio,'CB']/df_corp_bonds.loc[date,'Price'] ) * df_corp_bonds.loc[date,'Price']
        if df_portfolio_allocation.loc[portfolio,'PB'] >0:
            value_per_day += (df_portfolio_allocation.loc[portfolio,'PB']/df_pub_bonds.loc[date,'Price'] ) * df_pub_bonds.loc[date,'Price']
                
        if df_portfolio_allocation.loc[portfolio,'GO'] >0:
            value_per_day += (df_portfolio_allocation.loc[portfolio,'GO']/df_gold.loc[date,'Price'] ) * df_gold.loc[date,'Price']
                
        if df_portfolio_allocation.loc[portfolio,'CA'] >0:
            value_per_day += (df_portfolio_allocation.loc[portfolio,'CA']/df_cash.loc[date,'Price'] ) * df_cash.loc[date,'Price']

        values_per_day.append(value_per_day)
    volatility = std_dev(values_per_day)*100/statistics.mean(values_per_day)
    values_per_port.append(volatility)
print(values_per_port)





[6.2590559120883266e-15, 0.0, 7.428132754076226e-16, 0.0, 2.100993216784482e-15, 5.657097365528053e-15, 0.0, 0.0, 0.0, 7.428132754076226e-16, 2.100993216784482e-15, 0.0, 0.0, 0.0, 7.428132754076226e-16, 0.0, 0.0, 0.0, 1.9652991972859003e-15, 1.9652991972859003e-15, 1.050496608392241e-15, 0.0, 0.0, 0.0, 1.4856265508152452e-15, 2.100993216784482e-15, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9652991972859003e-15, 1.050496608392241e-15, 0.0, 0.0, 1.4856265508152452e-15, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.428132754076226e-16, 7.428132754076226e-16, 0.0, 0.0, 0.0, 3.40399806177351e-15, 3.40399806177351e-15, 3.40399806177351e-15, 5.608117245648724e-15, 0.0, 0.0, 0.0, 0.0, 2.100993216784482e-15, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9652991972859003e-15, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.428132754076226e-16, 7.428132754076226e-16, 7.428132754076226e-16, 7.428132754076226e-16, 7.428132754076226e-16, 7.428132754076226e-16, 0.0, 0.0, 0.0, 3.40399806177351e-15, 7